In [19]:
ticker = input('Please input the ticker of the stock: ')

In [114]:
class finance:
    def __init__(self,ticker):
        self.ticker = ticker
        
    def financial(self,IS=0,BS=0,CF=0):
        import requests
        from bs4 import BeautifulSoup
        import pandas as pd
        IS_keyword = ['Revenue','Operating Expenses','Income from Continuing Operations','Non-recurring Events','Net Income']
        BS_keyword = ['Assets','Liabilities','Equity']
        CF_keyword = ['Net Income','Operating Activities','Investing Activities','Financing Activities','Cash']
        link = 'https://finance.yahoo.com/quote/' + self.ticker + '/financials?p=' + self.ticker
        keyword = IS_keyword
        if BS:
            link = 'https://finance.yahoo.com/quote/' + self.ticker + '/balance-sheet?p=' + self.ticker
            keyword = BS_keyword
        if CF:
            link = 'https://finance.yahoo.com/quote/' + self.ticker + '/cash-flow?p=' + self.ticker
            keyword = CF_keyword
        response = requests.get(link)
        result_page = BeautifulSoup(response.content,'lxml')
        df_list = []
        fin_info = dict()
        col,data_col,data_frame = [None], [], []
        i = 1
        end = 0
        for tr_tag in result_page.find_all('tr'):
            for td_tag in tr_tag.find_all('td'):
                if td_tag.get('class') == ['C($gray)', 'Ta(end)']:
                    col.append(td_tag.get_text())
                if not td_tag.get('class') in [['Fw(b)', 'Fz(15px)'],['C($gray)', 'Ta(end)'],['Fw(b)', 'Fz(15px)', 'Pb(8px)', 'Pt(36px)']]:
                    if 1 <= i <= 5:
                        data_col.append(td_tag.get_text())
                        i += 1
                    else:
                        data_frame.append(data_col)
                        data_col = []
                        i = 1
                        data_col.append(td_tag.get_text())
                        i += 1
                if tr_tag.get('class') == ['Bdbw(0px)!', 'H(36px)']:
                    end += 1
                if end == 5:
                    data_frame.append(data_col)
                    df = pd.DataFrame(data_frame)
                    df_list.append(df)
                    end = 0
                    i = 1
                    data_col,data_frame = [], []
        for k in range(len(keyword)):
            col_name = col
            col_name[0] = keyword[k] + ' (All numbers in thousands)'
            df_list[k].columns = col_name
            df_list[k].set_index(col_name[0],inplace=True)
            fin_info[keyword[k]] = df_list[k]
        return (df_list, fin_info)
    
    def IS(self):
        for table in self.financial()[0]:
            print(table)
        return 'For separate tables, please use other attributes, e.g. "revenue".'
    
    def revenue(self):
        return self.financial()[1]['Revenue']
    
    def operating_expenses(self):
        return self.financial()[1]['Operating Expenses']
    
    def operating_income(self):
        return self.financial()[1]['Income from Continuing Operations']
    
    def non_recurring(self):
        return self.financial()[1]['Non-recurring Events']
    
    def net_income(self):
        return self.financial()[1]['Net Income']
    
    def assets(self):
        return self.financial(BS=1)[1]['Assets']
    
    def liabilities(self):
        return self.financial(BS=1)[1]['liabilities']
    
    def equity(self):
        return self.financial(BS=1)[1]['Equity']
    
    def CFO(self):
        return self.financial(CF=1)[1]['Operating Activities']
    
    def CFI(self):
        return self.financial(CF=1)[1]['Investing Activities']
    
    def CFF(self):
        return self.financial(CF=1)[1]['Financing Activities']
    
    def cash(self):
        return self.financial(CF=1)[1]['Cash']
    
    def complete_analysis(self):
        import requests
        from bs4 import BeautifulSoup
        import pandas as pd
        link = 'https://finance.yahoo.com/quote/' + self.ticker + '/analysis?p' + self.ticker
        response = requests.get(link)
        if response.status_code == 200:
            pass
        else:
            print('An error occurred when accessing the page!')
        result_page = BeautifulSoup(response.content,'lxml')
        df_list = []
        analysis_info = dict()
        for table_tag in result_page.find_all('table'):
            col,data_col,data_frame = [], [], []
            for th_tag in table_tag.find_all('th'):
                col.append(th_tag.get_text())
            i = 1
            for td_tag in table_tag.find_all('td'):
                if 1 <= i <= 5:
                    data_col.append(td_tag.get_text())
                    i += 1
                else:
                    data_frame.append(data_col)
                    data_col = []
                    i = 1
                    data_col.append(td_tag.get_text())
                    i += 1
            data_frame.append(data_col)
            df = pd.DataFrame(data_frame,columns = col)
            df.set_index(col[0],inplace=True)
            df_list.append(df)
            analysis_info[col[0]] = df
        return (df_list, analysis_info)
    
    def analysis(self):
        return self.complete_analysis()[0]
    
    def est_earnings(self):
        return self.complete_analysis()[1]['Earnings Estimate']
    
    def est_revenue(self):
        return self.complete_analysis()[1]['Revenue Estimate']
    
    def history_earnings(self):
        return self.complete_analysis()[1]['Earnings History']
    
    def EPS_trend(self):
        return self.complete_analysis()[1]['EPS Trend']
    
    def EPS_revision(self):
        return self.complete_analysis()[1]['EPS Revisions']
    
    def est_growth(self):
        return self.complete_analysis()[1]['Growth Estimates']
    
    def holders_structure(self):
        import requests
        from bs4 import BeautifulSoup
        import pandas as pd
        link = 'https://finance.yahoo.com/quote/' + self.ticker + '/holders?p' + self.ticker
        response = requests.get(link)
        result_page = BeautifulSoup(response.content,'lxml')
        df_list = []
        for div_tag in result_page.find_all('div'):
            if div_tag.get('class') in [['Mt(25px)', 'Ovx(a)', 'W(100%)'],['W(100%)', 'Mb(20px)']]:
                if div_tag.find('span').get_text() == 'Breakdown':
                    j = 2
                    col = ['Major Holders Breakdown',' ']
                else:
                    j = 5
                    col = []
                i = 1
                data_col,data_frame = [],[]
                for th_tag in div_tag.find_all('th'):
                    col.append(th_tag.get_text())
                for td_tag in div_tag.find_all('td'):
                    if 1 <= i <= j:
                        data_col.append(td_tag.get_text())
                        i += 1
                    else:
                        data_frame.append(data_col)
                        data_col = []
                        i = 1
                        data_col.append(td_tag.get_text())
                        i += 1
                data_frame.append(data_col)
                df = pd.DataFrame(data_frame,columns = col)
                df.set_index(col[0],inplace=True)
                df_list.append(df)
        df_list[1].index.names = ['Top Institutional Holders']
        df_list[2].index.names = ['Top Mutual Fund Holders']
        holders_info = {'major':df_list[0],'institutional':df_list[1],'mutual fund':df_list[2]}
        return (df_list,holders_info)
    
    def holders(self):
        return self.holders_structure()[0]
    
    def major_holders(self):
        return self.holders_structure()[1]['major']
    
    def ins_holders(self):
        return self.holders_structure()[1]['institutional']
    
    def mf_holders(self):
        return self.holders_structure()[1]['mutual fund']

In [50]:
# def holders(ticker):
#     import requests
#     from bs4 import BeautifulSoup
#     import pandas as pd
#     link = 'https://finance.yahoo.com/quote/' + ticker + '/holders?p' + ticker
#     response = requests.get(link)
#     result_page = BeautifulSoup(response.content,'lxml')
#     df_list = []
#     for div_tag in result_page.find_all('div'):
#         if div_tag.get('class') in [['Mt(25px)', 'Ovx(a)', 'W(100%)'],['W(100%)', 'Mb(20px)']]:
#             if div_tag.find('span').get_text() == 'Breakdown':
#                 j = 2
#                 col = ['Major Holders Breakdown',' ']
#             else:
#                 j = 5
#                 col = []
#             i = 1
#             data_col,data_frame = [],[]
#             for th_tag in div_tag.find_all('th'):
#                 col.append(th_tag.get_text())
#             for td_tag in div_tag.find_all('td'):
#                 if 1 <= i <= j:
#                     data_col.append(td_tag.get_text())
#                     i += 1
#                 else:
#                     data_frame.append(data_col)
#                     data_col = []
#                     i = 1
#                     data_col.append(td_tag.get_text())
#                     i += 1
#             data_frame.append(data_col)
#             df = pd.DataFrame(data_frame,columns = col)
#             df.set_index(col[0],inplace=True)
#             df_list.append(df)
#     df_list[1].index.names = ['Top Institutional Holders']
#     df_list[2].index.names = ['Top Mutual Fund Holders']
#     holders_info = {'major':df_list[0],'institutional':df_list[1],'mutual fund':df_list[2]}
#     return holders_info

In [62]:
# def financial(ticker,IS=0,BS=0,CF=0):
#     import requests
#     from bs4 import BeautifulSoup
#     import pandas as pd
#     IS_keyword = ['Revenue','Operating Expenses','Income from Continuing Operations','Non-recurring Events','Net Income']
#     BS_keyword = ['Assets','Liabilities','Equity']
#     CF_keyword = ['Net Income','Operating Activities','Investing Activities','Financing Activities','Exchange Rate Changes']
#     link = 'https://finance.yahoo.com/quote/' + ticker + '/financials?p=' + ticker
#     keyword = IS_keyword
#     if BS:
#         link = 'https://finance.yahoo.com/quote/' + ticker + '/balance-sheet?p=' + ticker
#         keyword = BS_keyword
#     if CF:
#         link = 'https://finance.yahoo.com/quote/' + ticker + '/cash-flow?p=' + ticker
#         keyword = CF_keyword
#     response = requests.get(link)
#     result_page = BeautifulSoup(response.content,'lxml')
#     df_list = []
#     fin_info = dict()
#     col,data_col,data_frame = [None], [], []
#     i = 1
#     end = 0
#     for tr_tag in result_page.find_all('tr'):
#         for td_tag in tr_tag.find_all('td'):
#             if td_tag.get('class') == ['C($gray)', 'Ta(end)']:
#                 col.append(td_tag.get_text())
#             if not td_tag.get('class') in [['Fw(b)', 'Fz(15px)'],['C($gray)', 'Ta(end)'],['Fw(b)', 'Fz(15px)', 'Pb(8px)', 'Pt(36px)']]:
#                 if 1 <= i <= 5:
#                     data_col.append(td_tag.get_text())
#                     i += 1
#                 else:
#                     data_frame.append(data_col)
#                     data_col = []
#                     i = 1
#                     data_col.append(td_tag.get_text())
#                     i += 1
#             if tr_tag.get('class') == ['Bdbw(0px)!', 'H(36px)']:
#                 end += 1
#             if end == 5:
#                 data_frame.append(data_col)
#                 df = pd.DataFrame(data_frame)
#                 df_list.append(df)
#                 end = 0
#                 i = 1
#                 data_col,data_frame = [], []
#     for k in range(len(keyword)):
#         col_name = col
#         col_name[0] = keyword[k] + ' (All numbers in thousands)'
#         df_list[k].columns = col_name
#         df_list[k].set_index(col_name[0],inplace=True)
#         fin_info[keyword[k]] = df_list[k]
#     return fin_info

In [115]:
a = finance('MS')

In [116]:
a.mf_holders()

,Shares,Date Reported,% Out,Value
Top Mutual Fund Holders,,,,
Vanguard Total Stock Market Index Fund,"32,672,577","Dec 30, 2017",1.87%,"1,720,211,228"
Vanguard 500 Index Fund,"23,782,460","Dec 30, 2017",1.36%,"1,252,146,555"
Price (T.Rowe) Blue Chip Growth Fund Inc.,"18,940,200","Jun 29, 2018",1.09%,"893,788,011"
Fidelity Contrafund Inc,"15,684,371","Jun 29, 2018",0.90%,"740,145,445"
SPDR S&P 500 ETF Trust,"15,213,627","Jun 29, 2018",0.87%,"717,931,037"
Vanguard Institutional Index Fund-Institutional Index Fund,"14,098,725","Dec 30, 2017",0.81%,"742,297,892"
Select Sector SPDR Fund-Financial,"12,623,012","Jun 29, 2018",0.72%,"595,679,918"
Fidelity 500 Index Fund,"8,972,779","Jun 29, 2018",0.51%,"423,425,428"
iShares Core S&P 500 ETF,"8,767,239","Jun 29, 2018",0.50%,"413,725,996"
